In [20]:
import mysql.connector
from mysql.connector import connect, Error
import pandas as pd


In [78]:
try:
  mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="seatTle*!31",
    database='art_auction'
  )
except Error as e:
  print(e)

mycursor = mydb.cursor()

In [32]:
show_db_query = "SHOW DATABASES"
with mydb.cursor() as cursor:
    cursor.execute(show_db_query)
    for db in cursor:
        print(db)

('art_auction',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [26]:
raw = pd.read_csv('../../data/christies_data.csv')
raw.columns

Index(['Unnamed: 0', 'object_id', 'lot_id_txt', 'event_type', 'start_date',
       'end_date', 'url', 'is_auction_over', 'is_in_progress',
       'title_primary_txt', 'title_secondary_txt', 'title_tertiary_txt',
       'consigner_information', 'description_txt', 'image', 'estimate_visible',
       'estimate_on_request', 'is_estimate_unknown', 'price_on_request',
       'estimate_low', 'estimate_high', 'estimate_txt', 'lot_withdrawn',
       'price_realised', 'price_realised_txt', 'current_bid',
       'current_bid_txt', 'is_saved', 'show_save', 'lot_title', 'location'],
      dtype='object')

In [28]:
string = ''
for c in raw.columns:
    string += str(c) + ', '
print(string)

Unnamed: 0, object_id, lot_id_txt, event_type, start_date, end_date, url, is_auction_over, is_in_progress, title_primary_txt, title_secondary_txt, title_tertiary_txt, consigner_information, description_txt, image, estimate_visible, estimate_on_request, is_estimate_unknown, price_on_request, estimate_low, estimate_high, estimate_txt, lot_withdrawn, price_realised, price_realised_txt, current_bid, current_bid_txt, is_saved, show_save, lot_title, location, 


In [29]:
print(raw.head(1))

   Unnamed: 0  object_id lot_id_txt event_type         start_date  \
0           0    5287253          1       Sale  2010-01-28T00:00Z   

            end_date                                                url  \
0  2010-01-28T05:00Z  https://www.christies.com/lot/lot-5287253?ldp_...   

   is_auction_over  is_in_progress title_primary_txt  ...  \
0             True           False    AN AGATE GEODE  ...   

          estimate_txt  lot_withdrawn price_realised price_realised_txt  \
0  USD 800 - USD 1,200          False         1875.0          USD 1,875   

  current_bid  current_bid_txt  is_saved  show_save  \
0         NaN              NaN     False       True   

                                           lot_title  location  
0  A Cabinet of Curiosities Selections from the P...  New York  

[1 rows x 31 columns]


In [132]:
# create a table for the raw data

create_table_query = """
DROP TABLE auction_raw;
CREATE TABLE IF NOT EXISTS auction_raw(
    pd_id INT,
    object_id INT,
    lot_id_txt INT,
    event_type VARCHAR(100),
    start_date DATETIME,
    end_date DATETIME,
    url VARCHAR(500),
    is_auction_over BOOL,
    is_in_progress BOOL,
    title_primary_txt VARCHAR(100),
    title_secondary_txt VARCHAR(100),
    title_tertiary_txt VARCHAR(100),
    consigner_information VARCHAR(100),
    description_txt VARCHAR(100),
    estimate_visible BOOL,
    estimate_on_request BOOL,
    is_estimate_unknown BOOL,
    price_on_request BOOL,
    estimate_low DOUBLE,
    estimate_high DOUBLE,
    estimate_txt VARCHAR(100),
    lot_withdrawn BOOL,
    price_realised DOUBLE,
    price_realised_txt VARCHAR(100),
    current_bid DOUBLE,
    current_bid_txt VARCHAR(100),
    is_saved BOOL,
    show_save BOOL,
    lot_title VARCHAR(500),
    location VARCHAR(100)
)
"""
mycursor.execute(create_table_query, multi=True)
mydb.commit()

In [121]:
df = raw.copy()
df = df.drop('image', axis=1)
#df['Unnamed: 0'] = (df['Unnamed: 0'].astype(object))
#df['object_id'] = (df['object_id'].astype(object))
for col in df.columns:
    if str(df[col].dtype) != 'object':
        df[col] = df[col].astype(object)
df['start_date'] = df['start_date'].str.replace('T', ' ').replace('Z', ':00')
df['end_date'] = df['end_date'].str.replace('T', ' ').replace('Z', ':00')
df['start_date'] = df['start_date'].str.replace('Z', ':00')
df['end_date'] = df['end_date'].str.replace('Z', ':00')
df['description_txt'] = df['description_txt'].str.slice(0,99)
df['title_secondary_txt'] = df['title_secondary_txt'].str.slice(0,99)
df['consigner_information'] = df['consigner_information'].str.slice(0,99)
df['title_primary_txt'] = df['title_primary_txt'].str.slice(0,99)

records = []
for i in range(len(df)):
    row = tuple(df.iloc[i])
    records.append(row)
print('-- ROWS COMPILED --')

-- ROWS COMPILED --


In [133]:

for batch in range(100,len(records),100):
    to_insert = records[batch:batch+99]
    insert_query = '''
                INSERT INTO auction_raw
                (pd_id, object_id, lot_id_txt, event_type, start_date, end_date, url, is_auction_over, is_in_progress, title_primary_txt, title_secondary_txt, title_tertiary_txt, consigner_information, description_txt, estimate_visible, estimate_on_request, is_estimate_unknown, price_on_request, estimate_low, estimate_high, estimate_txt, lot_withdrawn, price_realised, price_realised_txt, current_bid, current_bid_txt, is_saved, show_save, lot_title, location)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                '''

    mycursor.executemany(insert_query,to_insert)
    mydb.commit()

DatabaseError: 1265 (01000): Data truncated for column 'lot_id_txt' at row 50